In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

In [6]:
bcf = pd.read_excel("BCF.xlsx")
ps_bart = pd.read_excel("ps_BART.xlsx")

In [7]:
bcf.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [8]:
# Calculate the mean and standard deviation for each column
mean_values = bcf.mean()
std_values = bcf.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.551907
empirical_ATE_MAE         0.410245
empirical_ATE_MAPE        0.230059
empirical_ATE_len         0.785081
empirical_ATE_cover       0.531280
theoretical_CATE_RMSE     2.108224
theoretical_CATE_MAE      1.714231
theoretical_CATE_MAPE     0.227346
theoretical_CATE_len      2.971460
theoretical_CATE_cover    0.476029
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.079689
empirical_ATE_MAE         0.030674
empirical_ATE_MAPE        0.018567
empirical_ATE_len         0.085368
empirical_ATE_cover       0.096845
theoretical_CATE_RMSE     1.080347
theoretical_CATE_MAE      0.911852
theoretical_CATE_MAPE     0.042670
theoretical_CATE_len      1.162028
theoretical_CATE_cover    0.136811
dtype: float64


In [9]:
bcf.head()

,empirical_ATE_RMSE,empirical_ATE_MAE,empirical_ATE_MAPE,empirical_ATE_len,empirical_ATE_cover,theoretical_CATE_RMSE,theoretical_CATE_MAE,theoretical_CATE_MAPE,theoretical_CATE_len,theoretical_CATE_cover
0,0.563575,0.423316,0.228946,0.735728,0.438,1.512525,1.209765,0.207956,2.488958,0.589496
1,0.498533,0.369064,0.207379,0.820735,0.686,1.623359,1.309866,0.174101,3.344776,0.759752
2,0.549044,0.479645,0.289082,0.749773,0.334,1.475697,1.181576,0.158502,2.828600,0.677120
3,0.529102,0.426903,0.250314,0.697705,0.500,1.016132,0.808092,0.133110,2.422582,0.835920
4,0.623455,0.425578,0.220585,0.865900,0.582,2.070402,1.674893,0.232119,3.593977,0.610144


In [10]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
bcf_SE_empirical_ATE_cover = (((0.95 - bcf['empirical_ATE_cover'])**2))
bcf_AE_empirical_ATE_cover = (np.abs(0.95 - bcf['empirical_ATE_cover']))

bcf_SE_theoretical_CATE_cover = (((0.95 - bcf['theoretical_CATE_cover'])**2))
bcf_AE_theoretical_CATE_cover = (np.abs(0.95 - bcf['theoretical_CATE_cover']))

In [11]:
ps_bart.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [12]:
# Calculate the mean and standard deviation for each column
mean_values = ps_bart.mean()
std_values = ps_bart.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.315523
empirical_ATE_MAE         0.250886
empirical_ATE_MAPE        0.149301
empirical_ATE_len         1.572020
empirical_ATE_cover       0.988340
theoretical_CATE_RMSE     0.913717
theoretical_CATE_MAE      0.711927
theoretical_CATE_MAPE     0.123243
theoretical_CATE_len      3.661778
theoretical_CATE_cover    0.970354
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.113608
empirical_ATE_MAE         0.109127
empirical_ATE_MAPE        0.071191
empirical_ATE_len         0.169757
empirical_ATE_cover       0.036157
theoretical_CATE_RMSE     0.714670
theoretical_CATE_MAE      0.578251
theoretical_CATE_MAPE     0.028239
theoretical_CATE_len      1.677733
theoretical_CATE_cover    0.037919
dtype: float64


In [13]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
ps_bart_SE_empirical_ATE_cover = (((0.95 - ps_bart['empirical_ATE_cover'])**2))
ps_bart_AE_empirical_ATE_cover = (np.abs(0.95 - ps_bart['empirical_ATE_cover']))

ps_bart_SE_theoretical_CATE_cover = (((0.95 - ps_bart['theoretical_CATE_cover'])**2))
ps_bart_AE_theoretical_CATE_cover = (np.abs(0.95 - ps_bart['theoretical_CATE_cover']))

In [14]:
# Function to check normality using Shapiro-Wilk test
def check_normality(data):
    stat, p = stats.shapiro(data)
    return p > 0.05  # If p > 0.05, data is normally distributed

# Function to check homoscedasticity using Levene's test
def check_homoscedasticity(data1, data2):
    stat, p = stats.levene(data1, data2)
    return p > 0.05  # If p > 0.05, variances are equal



# ATE RMSE

In [15]:
i=0
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_RMSE
Mann-Whitney U Test:
Statistic: 626.0, p-value: 1.1816086154319694e-26
Wilcoxon Signed-Rank Test:
Statistic: 32.0, p-value: 1.0192962404545369e-17
Kruskal-Wallis H Test:
Statistic: 114.22015522388062, p-value: 1.1661507579579673e-26
Levene's Test:
Statistic: 7.009649987416477, p-value: 0.008760064180883232
Brown-Forsythe Test:
Statistic: 7.009649987416477, p-value: 0.008760064180883232


# ATE MAE

In [16]:
i=1
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAE
Mann-Whitney U Test:
Statistic: 1090.0, p-value: 1.2669060915642081e-21
Wilcoxon Signed-Rank Test:
Statistic: 179.0, p-value: 7.245464848115237e-16
Kruskal-Wallis H Test:
Statistic: 91.27223880597012, p-value: 1.2520493130294936e-21
Levene's Test:
Statistic: 52.89314646027194, p-value: 8.012787879361972e-12
Brown-Forsythe Test:
Statistic: 52.89314646027194, p-value: 8.012787879361972e-12


# ATE MAPE

In [17]:
i=2
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAPE
Mann-Whitney U Test:
Statistic: 1491.0, p-value: 1.0114815102119759e-17
Wilcoxon Signed-Rank Test:
Statistic: 377.0, p-value: 1.5185669321220046e-13
Kruskal-Wallis H Test:
Statistic: 73.51093134328357, p-value: 1.00080367651597e-17
Levene's Test:
Statistic: 52.5446857839053, p-value: 9.217736941274675e-12
Brown-Forsythe Test:
Statistic: 52.5446857839053, p-value: 9.217736941274675e-12


# CATE RMSE

In [18]:
i=5
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_RMSE
Mann-Whitney U Test:
Statistic: 789.0, p-value: 7.990910821251365e-25
Wilcoxon Signed-Rank Test:
Statistic: 2.0, p-value: 4.139373908714419e-18
Kruskal-Wallis H Test:
Statistic: 105.8657970149253, p-value: 7.890178405026158e-25
Levene's Test:
Statistic: 9.179096681885099, p-value: 0.0027744820895215096
Brown-Forsythe Test:
Statistic: 9.179096681885099, p-value: 0.0027744820895215096


# CATE MAE

In [19]:
i=6
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAE
Mann-Whitney U Test:
Statistic: 718.0, p-value: 1.2996343516475128e-25
Wilcoxon Signed-Rank Test:
Statistic: 1.0, p-value: 4.0161556854627e-18
Kruskal-Wallis H Test:
Statistic: 109.46581492537314, p-value: 1.2829817760399448e-25
Levene's Test:
Statistic: 11.11803072871698, p-value: 0.0010205319751469585
Brown-Forsythe Test:
Statistic: 11.11803072871698, p-value: 0.0010205319751469585


# CATE MAPE

In [20]:
i=7
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAPE
Mann-Whitney U Test:
Statistic: 225.0, p-value: 1.9024444786543233e-31
Wilcoxon Signed-Rank Test:
Statistic: 3.0, p-value: 4.2663227307182344e-18
Kruskal-Wallis H Test:
Statistic: 136.12313432835822, p-value: 1.875327684927417e-31
Levene's Test:
Statistic: 14.453488573649718, p-value: 0.00019123314110614743
Brown-Forsythe Test:
Statistic: 14.453488573649718, p-value: 0.00019123314110614743


# ATE len

In [21]:
i=3
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_len
Mann-Whitney U Test:
Statistic: 10000.0, p-value: 2.562143669163401e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 149.25373134328356, p-value: 2.523939423990366e-34
Levene's Test:
Statistic: 26.885767249343193, p-value: 5.310706935883457e-07
Brown-Forsythe Test:
Statistic: 26.885767249343193, p-value: 5.310706935883457e-07


# CATE len

In [22]:
i=8
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_len
Mann-Whitney U Test:
Statistic: 6818.0, p-value: 8.960036557042221e-06
Wilcoxon Signed-Rank Test:
Statistic: 1.0, p-value: 4.0161556854627e-18
Kruskal-Wallis H Test:
Statistic: 19.732083582089558, p-value: 8.909300816270065e-06
Levene's Test:
Statistic: 0.7306694794326977, p-value: 0.39369936150447227
Brown-Forsythe Test:
Statistic: 0.7306694794326977, p-value: 0.39369936150447227


# ATE Cover SE

In [23]:
group1 = np.array(ps_bart_SE_empirical_ATE_cover)
group2 = np.array(bcf_SE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover SE
ATE Cover RMSE for ps-BART: 0.05257565976761493 and BCF: 0.42966461339049083
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.404178461800673e-35
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.89392293135041e-18
Kruskal-Wallis H Test:
Statistic: 153.95648833760256, p-value: 2.367216101973846e-35
Levene's Test:
Statistic: 258.0458283070509, p-value: 1.0071121841155547e-37
Brown-Forsythe Test:
Statistic: 258.0458283070509, p-value: 1.0071121841155547e-37


# ATE Cover AE

In [24]:
group1 = np.array(ps_bart_AE_empirical_ATE_cover)
group2 = np.array(bcf_AE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover AE
ATE Cover MAE for ps-BART: 0.04822000000000003 and BCF: 0.41872
Mann-Whitney U Test:
Statistic: 0.0, p-value: 2.404178461800673e-35
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.889092982277817e-18
Kruskal-Wallis H Test:
Statistic: 153.95648833760256, p-value: 2.367216101973846e-35
Levene's Test:
Statistic: 192.7561445771467, p-value: 4.736778117105011e-31
Brown-Forsythe Test:
Statistic: 192.7561445771467, p-value: 4.736778117105011e-31


# CATE Cover SE

In [25]:
group1 = np.array(ps_bart_SE_theoretical_CATE_cover)
group2 = np.array(bcf_SE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover SE
CATE Cover RMSE for ps-BART: 0.04286922889346159 and BCF: 0.493131356065866
Mann-Whitney U Test:
Statistic: 13.0, p-value: 3.7841390489101244e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.47884361935587, p-value: 3.727857215313555e-34
Levene's Test:
Statistic: 160.6911265972029, p-value: 2.3839578016896008e-27
Brown-Forsythe Test:
Statistic: 160.6911265972029, p-value: 2.3839578016896008e-27


# CATE Cover AE

In [26]:
group1 = np.array(ps_bart_AE_theoretical_CATE_cover)
group2 = np.array(bcf_AE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover AE
CATE Cover MAE for ps-BART: 0.03701576000000002 and BCF: 0.4739709599999999
Mann-Whitney U Test:
Statistic: 13.0, p-value: 3.7841390489101244e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.47884361935587, p-value: 3.727857215313555e-34
Levene's Test:
Statistic: 88.14974090621716, p-value: 1.48288597852058e-17
Brown-Forsythe Test:
Statistic: 88.14974090621716, p-value: 1.48288597852058e-17
